In [1]:
!pip install efficientnet

In [2]:
import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
import efficientnet.tfkeras as efn
from keras.applications.densenet import DenseNet201
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


##  TPU or GPU detection

In [3]:
AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [4]:
# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Configuration
EPOCHS = 40
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IM_Z = 768

In [5]:
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

In [6]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
sub = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')

train_paths = train.image_id.apply(format_path).values
test_paths = test.image_id.apply(format_path).values

train_labels = train.loc[:, 'healthy':].values

train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, train_labels, test_size=0.1, random_state=2020)

In [7]:
def decode_image(filename, label=None, image_size=(IM_Z, IM_Z)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
#     image = tf.image.adjust_brightness(image, delta=0.2)
#     image = tf.image.adjust_contrast(image,2)
    
    if label is None:
        return image
    else:
        return image, label

In [8]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .cache()
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

In [9]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, 
               lr_min=0.000001, lr_rampup_epochs=20, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

ch_p = ModelCheckpoint(filepath="model_ef.h5", monitor='val_loss', save_weights_only=True,
                                                 verbose=1)

In [10]:
with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB7(
            input_shape=(IM_Z, IM_Z, 3),
            weights='imagenet',
            include_top=False
        ),
        L.GlobalAveragePooling2D(),
        L.Dense(train_labels.shape[1], activation='softmax')
    ])
        
    model.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
#     model.summary()

258441216/258434480 [==============================] - 8s 0us/step


In [11]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE 

In [12]:
history = model.fit(
    train_dataset, 
    epochs=EPOCHS, 
    callbacks=[lr_schedule, ch_p],
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_dataset
)

Train for 25 steps, validate for 3 steps

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/40
24/25 [===========================>..] - ETA: 14s - loss: 1.3568 - categorical_accuracy: 0.3568
Epoch 00001: saving model to model_ef.h5
25/25 [==============================] - 413s 17s/step - loss: 1.3553 - categorical_accuracy: 0.3619 - val_loss: 1.4648 - val_categorical_accuracy: 0.2623

Epoch 00002: LearningRateScheduler reducing learning rate to 3.95e-05.
Epoch 2/40
24/25 [===========================>..] - ETA: 1s - loss: 1.1406 - categorical_accuracy: 0.6582
Epoch 00002: saving model to model_ef.h5
25/25 [==============================] - 32s 1s/step - loss: 1.1315 - categorical_accuracy: 0.6644 - val_loss: 1.1136 - val_categorical_accuracy: 0.5792

Epoch 00003: LearningRateScheduler reducing learning rate to 6.9e-05.
Epoch 3/40
24/25 [===========================>..] - ETA: 1s - loss: 0.5654 - categorical_accuracy: 0.8548
Epoch 00003: saving model to model_ef.

In [13]:
with strategy.scope():
    model1 = tf.keras.Sequential([
        tf.keras.applications.DenseNet201(
            input_shape=(IM_Z, IM_Z, 3),
            weights='imagenet',
            include_top=False
        ),
        L.GlobalAveragePooling2D(),
        L.Dense(train_labels.shape[1], activation='softmax')
    ])
        
             
    model1.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
#     model.summary()

74842112/74836368 [==============================] - 3s 0us/step


In [14]:
ch_p_den = ModelCheckpoint(filepath="model_den.h5", monitor='val_loss', save_weights_only=True,
                                                 verbose=1)

In [15]:
history1 = model1.fit(
    train_dataset, 
    epochs=EPOCHS, 
    callbacks=[lr_schedule, ch_p_den],
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_dataset
)

Train for 25 steps, validate for 3 steps

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/40
24/25 [===========================>..] - ETA: 10s - loss: 1.3612 - categorical_accuracy: 0.3587
Epoch 00001: saving model to model_den.h5
25/25 [==============================] - 343s 14s/step - loss: 1.3558 - categorical_accuracy: 0.3612 - val_loss: 1.3310 - val_categorical_accuracy: 0.4645

Epoch 00002: LearningRateScheduler reducing learning rate to 3.95e-05.
Epoch 2/40
24/25 [===========================>..] - ETA: 0s - loss: 0.7023 - categorical_accuracy: 0.7643
Epoch 00002: saving model to model_den.h5
25/25 [==============================] - 22s 889ms/step - loss: 0.6874 - categorical_accuracy: 0.7681 - val_loss: 0.7804 - val_categorical_accuracy: 0.6612

Epoch 00003: LearningRateScheduler reducing learning rate to 6.9e-05.
Epoch 3/40
24/25 [===========================>..] - ETA: 0s - loss: 0.3682 - categorical_accuracy: 0.8835
Epoch 00003: saving model to mode

In [16]:
# model.load_weights("model_ef.h5")
# model.load_weights("model_den.h5")

In [17]:
# probs = model.predict(test_dataset, verbose=1)
probs = (model1.predict(test_dataset)+model.predict(test_dataset))/2

In [18]:
sub.loc[:, 'healthy':] = probs
sub.to_csv('submission.csv', index=False)
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,1.293858e-05,2.599403e-03,0.997371,0.000016
1,Test_1,7.660828e-05,6.128404e-05,0.999833,0.000029
2,Test_2,2.546491e-07,3.763144e-07,0.000002,0.999998
3,Test_3,9.998404e-01,1.545112e-06,0.000066,0.000092
4,Test_4,8.315535e-06,1.649597e-03,0.998317,0.000026
